## Importing libraries

In [126]:
# Giving a seed value.Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps that are neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

# 5. Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

#Import all the necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import time
from matplotlib import pyplot

#Access all the necessary variables and datasets
#random_forest_data has all the chemistry compositions and microalloy elements compositions and output column been modified.
%store -r random_forest_data 
%store -r top_features_data
rows,cols=random_forest_data.shape #161 statistical features and one output column
top_features_data #coming from random forest code

%store -r feature_imp_RF 
#using feature_imp_RF from Random_Forest_Code which is a dataframe containing features and its corresponding importance score.

total_features=feature_imp_RF["Feature"] #considering all 161 features from random forest
total_features_list= total_features.tolist()

total_features.shape

GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


(161,)

In [127]:
random_forest_data

,SPEEDMEAN,STOPPERMEAN,TUNDISHWEIGHTMEAN,TUNDISHTEMPERATUREMEAN,TUNDISHTEMPERATUREMIN,TUNDISHTEMPERATUREMAX,BROADFACE1WATERFLOWMEAN,BROADFACE1WATERFLOWVARIANCE,BROADFACE2WATERFLOWMEAN,BROADFACE2WATERFLOWVARIANCE,...,PER_ALCMEAN,PER_ALNMEAN,PER_NBCMEAN,PER_VNMEAN,PER_TINMEAN,PER_TICMEAN,PER_MNSMEAN,PER_VCMEAN,PER_NBNMEAN,OUTPUT
0,1.120270,50.758621,23.854885,1554.574713,1554.0,1555.0,3826.942529,5.341525,3820.425287,6.186947,...,0.002079,0.000137,0.001267,0.000024,0.000043,0.000651,0.002926,0.000371,0.000083,1
1,1.114253,56.146465,23.548081,1548.237374,1548.0,1549.0,3823.025253,9.519564,3823.176768,11.448551,...,0.002678,0.000180,0.001382,0.000026,0.000053,0.000788,0.003059,0.000387,0.000093,1
2,1.198736,61.086207,24.320115,1551.000000,1551.0,1551.0,3821.913793,6.366132,3821.626437,10.486887,...,0.002037,0.000163,0.001260,0.000028,0.000048,0.000595,0.003990,0.000350,0.000101,0
3,1.198899,65.455696,24.275696,1547.107595,1546.0,1548.0,3826.860759,8.828713,3826.151899,6.850344,...,0.001939,0.000144,0.001344,0.000026,0.000048,0.000651,0.002660,0.000350,0.000100,0
4,1.141729,66.005650,24.563785,1553.090395,1553.0,1554.0,3823.536723,9.514188,3823.525424,10.430145,...,0.002113,0.000151,0.001285,0.000027,0.000043,0.000607,0.003725,0.000378,0.000092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,0.983690,71.732143,23.541369,1545.142857,1545.0,1546.0,4000.803571,6.741178,4000.636905,9.136019,...,0.006482,0.000170,0.000421,0.000013,0.000010,0.000366,0.003682,0.000513,0.000011,0
587,0.795487,69.920635,27.779471,1555.682540,1554.0,1558.0,3958.724868,15125.331710,3966.460317,15426.523558,...,0.006992,0.000253,0.000426,0.000239,0.000004,0.000122,0.004050,0.006597,0.000015,1
588,0.911294,73.304124,22.814948,1537.123711,1536.0,1538.0,4052.396907,28.796073,4024.128866,11.359682,...,0.009100,0.000260,0.005642,0.000026,0.000081,0.002839,0.018383,0.000910,0.000161,1
589,0.918199,67.461165,24.742087,1543.466019,1542.0,1545.0,4030.233010,223.101046,4027.310680,8.719012,...,0.008918,0.000255,0.005460,0.000025,0.000076,0.002657,0.016292,0.000874,0.000156,1


## Defining the function for training

In [128]:
def training_data(new_features):
    # Creating a new data
    new_data = random_forest_data.loc[:,new_features] # Taking the features given by forward selection algorithm.
    new_data = pd.concat([new_data, random_forest_data['OUTPUT']], axis=1)  # Add the 'OUTPUT' column
    new_data = shuffle(new_data, random_state=42) #Shuffling the data along with the output column.
    
    #Taking all independent variables from shuffled new_data in X
    X=new_data.iloc[:,:-1]

    # Splitting the shuffled new_data into train and test
    new_total_length = X.shape[0]
    new_train_size = int(X.shape[0] * 0.8)
    new_test_size = X.shape[0] - new_train_size

    new_train_X = X.iloc[:new_train_size, :].to_numpy().astype(np.float32)  # 2D array
    new_train_Y = new_data['OUTPUT'][:new_train_size].to_numpy().astype(np.float32)  # 1D array

    new_test_X = X.iloc[new_train_size:, :].to_numpy().astype(np.float32)  # 2D array
    new_test_Y = new_data['OUTPUT'][new_train_size:].to_numpy().astype(np.float32)  # 1D array

    # Scaling the data
    scaler = RobustScaler()
    new_train_X = scaler.fit_transform(new_train_X)
    new_test_X = scaler.transform(new_test_X)

    # Create a new model with the updated feature set (Random Forest Classifier)
    #new_model = RandomForestClassifier()
    new_model = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2,
                                       min_samples_leaf=1, max_features="auto")

    # Train the new model
    new_model.fit(new_train_X, new_train_Y)

    # Evaluate the new model
    new_test_pred = new_model.predict(new_test_X)
    new_accuracy = accuracy_score(new_test_Y, new_test_pred)

    return new_accuracy

## Defining the forward selection algorithm 

In [129]:
def forward_selection():
    # Start the timer
    start_time = time.time()

    selected_features = []  # Starting with an empty feature set
    selected_features_accuracy = []  # For storing accuracy of selected feature
    condition = False
    count = 0  # To know how many times the loop runs
    accuracy = -np.inf  # Starting with a very low accuracy

    while not condition:
        remaining_features = [feature for feature in total_features_list if feature not in selected_features]

        temp_accuracy_store = []
        temp_feature_array = []

        for feature in remaining_features:
            # Add one feature to the feature set
            new_features = selected_features + [feature]
            new_accuracy = training_data(new_features)
            temp_accuracy_store.append(new_accuracy)
            temp_feature_array.append(feature)

        maximum_accuracy = max(temp_accuracy_store)

        # Check for convergence. Stop when no improvement.
        if maximum_accuracy <= accuracy:
            condition = True
            break

        # Update the baseline feature set if the accuracy score improved
        else:
            max_idx = np.argmax(temp_accuracy_store)
            accuracy = maximum_accuracy  # Update the accuracy
            selected_features.append(temp_feature_array[max_idx])  # Update the selected features
            selected_features_accuracy.append(accuracy)
            count += 1

    # Stop the timer and calculate the elapsed time
    end_time = time.time()
    training_time = end_time - start_time
    return selected_features, selected_features_accuracy, training_time

In [130]:
selected_features, selected_features_accuracy, training_time = forward_selection()

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

...

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

In [131]:
selected_features

['MOLDWIDTHTOPMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN',
 'PER_VCMEAN',
 'DAYSAFTERCIRCLARDMEAN',
 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN']

In [132]:
selected_features[-1]

'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN'

In [133]:
selected_features_accuracy[-1]

0.8487394957983193

In [134]:
selected_features_accuracy[-1]-selected_features_accuracy[-1]*0.12

0.746890756302521

## Adding again features to selected features to see if accuracy is around 81% for any other variable too.

In [135]:
def forward_selection_again():
    # Start the timer
    start_time = time.time()
     
    remaining_features = [feature for feature in total_features_list if feature not in selected_features]
    
    selected_features_copy=selected_features.copy()
    selected_features_accuracy_copy= selected_features_accuracy.copy()

    for feature in remaining_features:
        # Add one feature to the feature set
        new_features = selected_features + [feature]
        print(new_features)
        new_accuracy = training_data(new_features)
        if 0.79 < new_accuracy <=selected_features_accuracy[-1]:
            selected_features_copy.append(feature)
            selected_features_accuracy_copy.append(new_accuracy)

    # Stop the timer and calculate the elapsed time
    end_time = time.time()
    training_time = end_time - start_time
    return selected_features_copy, selected_features_accuracy_copy, training_time

In [136]:
selected_features_copy, selected_features_accuracy_copy, training_time = forward_selection_again()

['MOLDWIDTHTOPMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'PER_VCMEAN', 'DAYSAFTERCIRCLARDMEAN', 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN', 'STEELLEVELINMOLD-SETMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'PER_VCMEAN', 'DAYSAFTERCIRCLARDMEAN', 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN', 'NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'PER_VCMEAN', 'DAYSAFTERCIRCLARDMEAN', 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN', 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN']


...

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'PER_VCMEAN', 'DAYSAFTERCIRCLARDMEAN', 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN', 'SEG7+8TOPSECONDARYCOOLINGWATERFLOWSETPOINTMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'PER_VCMEAN', 'DAYSAFTERCIRCLARDMEAN', 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN', 'MOLDNARROWFACECOOLINGWATERINLETPRESSMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [137]:
len(selected_features_copy)

28

In [139]:
selected_features_copy

['MOLDWIDTHTOPMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN',
 'PER_VCMEAN',
 'DAYSAFTERCIRCLARDMEAN',
 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'DAYSAFTERSHUTDOWNMEAN',
 'MOLDTAPERPERCENTMEAN',
 'DAYSAFTERSPRAYCLEANINGMEAN',
 'PER_CMEAN',
 'STEELLEVELINMOLD-ACTMEAN',
 'MOLDBROADFACECOOLINGWATERINLETTEMPMEAN',
 'BROADFACE2HEATFLUXMAX',
 'TUNDISHTEMPERATUREMEAN',
 'DAYSAFTERNARROWFACEALIGNMENTMEAN',
 'BROADFACE2HEATFLUXMEAN',
 'PER_VMEAN',
 'NARROWFACE1WATERFLOWMIN',
 'BROADFACE1HEATFLUXMIN',
 'TUNDISHTEMPERATUREMIN',
 'MOLDNARROWFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'NARROWFACE2HEATFLUXMIN',
 'SEG0ASECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDNARROWFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'PER_CAMEAN']

In [138]:
selected_features_accuracy_copy

[0.6722689075630253,
 0.7647058823529411,
 0.7815126050420168,
 0.8235294117647058,
 0.8487394957983193,
 0.8235294117647058,
 0.8151260504201681,
 0.8235294117647058,
 0.7983193277310925,
 0.8067226890756303,
 0.8067226890756303,
 0.7983193277310925,
 0.7983193277310925,
 0.7983193277310925,
 0.7983193277310925,
 0.8067226890756303,
 0.7983193277310925,
 0.8151260504201681,
 0.8235294117647058,
 0.8067226890756303,
 0.7983193277310925,
 0.8151260504201681,
 0.7983193277310925,
 0.7983193277310925,
 0.8235294117647058,
 0.8151260504201681,
 0.8067226890756303,
 0.7983193277310925]